install

In [ ]:
!pip install transformers

import library

In [1]:
import pandas

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
nltk.download('punkt_tab')
nltk.download('punkt')
nltk.download('stopwords')
from transformers import AutoTokenizer, AutoModelForSequenceClassification

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [53]:
df = pandas.read_csv('/content/data.csv')
df.head(5)

,Sentence,Sentiment
0,The GeoSolutions technology will leverage Bene...,positive
1,"$ESI on lows, down $1.50 to $2.50 BK a real po...",negative
2,"For the last quarter of 2010 , Componenta 's n...",positive
3,According to the Finnish-Russian Chamber of Co...,neutral
4,The Swedish buyout firm has sold its remaining...,neutral


In [54]:
# df['Label'] = df['Label'].replace({0: 'negative', 1: 'positive', 2: 'neutral'})

df.head(5)

,Sentence,Sentiment
0,The GeoSolutions technology will leverage Bene...,positive
1,"$ESI on lows, down $1.50 to $2.50 BK a real po...",negative
2,"For the last quarter of 2010 , Componenta 's n...",positive
3,According to the Finnish-Russian Chamber of Co...,neutral
4,The Swedish buyout firm has sold its remaining...,neutral


In [ ]:
def preprocess_text(text):
  """
  Preprocesses English text.

  Args:
      text (str): The text to preprocess.

  Returns:
      str: The preprocessed text.
  """

  # 1. Remove URLs and special characters
  text = re.sub(r'http\S+', '', text) # Remove URLs
  text = re.sub(r'[^a-zA-Z\s]', '', text) # Remove special characters
  text = re.sub(r'\s+', ' ', text).strip()
  # 2. Convert to lowercase
  # text = text.lower()

  # 3. Tokenization
  words = nltk.word_tokenize(text)

  # 4. Remove stop words
  stop_words = set(stopwords.words('english'))
  words = [word for word in words if word not in stop_words]

  # 5. Join words back into text
  preprocessed_text = " ".join(words)

  return preprocessed_text

In [ ]:
df['preprocessed_text'] = df['body_text'].apply(preprocess_text)
df.head()

,Unnamed: 0,ticker,publish_date,publish_time,title,body_text,url,neg,neu,pos,compound,preprocessed_text
0,0,NVDA,2025-03-27,08:50,"Trump auto tariffs announcement, Lululemon to ...",Investing.com - U.S. stock futures point to a ...,https://uk.investing.com/news/economy-news/tru...,0.045,0.877,0.078,0.9898,Investingcom US stock futures point mixed open...
1,1,NVDA,2025-03-27,12:02,Wall Street analyst lists 3 reasons why you sh...,Risk Disclosure: Trading in financial instrume...,https://uk.investing.com/news/pro/wall-street-...,0.076,0.893,0.032,-0.8437,Risk Disclosure Trading financial instruments ...
2,2,NVDA,2025-03-27,09:32,AMD downgraded: Nvidia’s AI chips still hold ’...,Investing.com -- Jefferies on Thursday downgra...,https://uk.investing.com/news/stock-market-new...,0.042,0.855,0.103,0.9704,Investingcom Jefferies Thursday downgraded Adv...
3,3,NVDA,2025-03-27,09:52,Buy Nvidia shares as valuation is ’still compe...,Investing.com -- Bank of America (NYSE:) reaff...,https://uk.investing.com/news/stock-market-new...,0.045,0.854,0.101,0.9772,Investingcom Bank America NYSE reaffirmed Buy ...
4,4,NVDA,2025-03-27,08:41,FTSE 100 Live: London blue-chips escape worst ...,"FTSE 100 falls 29 points to 8,661\n\nNext repo...",https://uk.investing.com/news/stock-market-new...,0.049,0.858,0.093,0.9996,FTSE falls points Next reports first bn profit...


In [55]:
df

,Sentence,Sentiment
0,The GeoSolutions technology will leverage Bene...,positive
1,"$ESI on lows, down $1.50 to $2.50 BK a real po...",negative
2,"For the last quarter of 2010 , Componenta 's n...",positive
3,According to the Finnish-Russian Chamber of Co...,neutral
4,The Swedish buyout firm has sold its remaining...,neutral
...,...,...
5837,RISING costs have forced packaging producer Hu...,negative
5838,Nordic Walking was first used as a summer trai...,neutral
5839,"According shipping company Viking Line , the E...",neutral
5840,"In the building and home improvement trade , s...",neutral


In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2381 entries, 0 to 2380
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Label             2381 non-null   object
 1   Ticker            2381 non-null   object
 2   Headline          2381 non-null   object
 3   argmax_sentiment  2381 non-null   object
dtypes: object(4)
memory usage: 74.5+ KB


In [ ]:
# prompt: count neu col > 0.5

# Assuming 'neu' column exists in your DataFrame 'df'
neu_count = len(df[df['compound'] > 0.5])
print(f"Number of rows where 'neu' is greater than 0.5: {neu_count}")


Number of rows where 'neu' is greater than 0.5: 2096


In [68]:
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")

model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

In [77]:
# prompt: df tor andom 20 row

import random

# Sample 20 random rows from the DataFrame
random_rows = df.sample(n=500)

# Print the sampled rows
random_rows


,Sentence,Sentiment
1653,Finnish food industry company L+ñnnen Tehtaat ...,neutral
3057,In June it sold a 30 percent stake to Nordstje...,neutral
3776,"According to Seikku , the retail sector in Fin...",neutral
5311,Besides we have increased the share of meat in...,positive
427,CORPORATE IT UPDATE - ( C ) 1995-2009 M2 COMMU...,neutral
...,...,...
5674,Operating profit before non-recurring items wa...,negative
5252,The order also includes extensive maintenance ...,neutral
2458,Japan's Asahi to submit bid next week for SABM...,positive
1263,"$AAPL AAPL: Gundlach Slams iPad mini, Sees Dow...",negative


In [78]:
import numpy as np
import torch

def predict_sentiment(text):
    inputs = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    outputs = model(**inputs)
    prediction = torch.nn.functional.softmax(outputs.logits, dim=-1)
    print(outputs)

    return prediction

random_rows['sentiment_probs'] = random_rows['Sentence'].apply(predict_sentiment)

# Example to access probabilities for the first article
first_article_probs = random_rows['sentiment_probs'].iloc[0]

# Accessing individual probabilities (assuming 3 classes: positive, negative, neutral)
# positive_prob = first_article_probs[0][0]
# negative_prob = first_article_probs[0][1]
# neutral_prob = first_article_probs[0][2]


SequenceClassifierOutput(loss=None, logits=tensor([[-0.9330, -0.7943,  2.2351]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
SequenceClassifierOutput(loss=None, logits=tensor([[-0.0115, -2.1819,  2.3957]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
SequenceClassifierOutput(loss=None, logits=tensor([[-1.2664, -0.3155,  2.3459]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
SequenceClassifierOutput(loss=None, logits=tensor([[ 2.0238, -2.5552, -0.7066]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
SequenceClassifierOutput(loss=None, logits=tensor([[-0.2650, -1.5967,  2.2293]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
SequenceClassifierOutput(loss=None, logits=tensor([[ 2.0660, -2.3133, -1.1270]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
SequenceClassifierOutput(loss=None, logits=tensor([[ 0.1823, -2.3154,  2.2544]], grad_fn=<AddmmBackward0>), hidden_states=None, at

In [71]:
# prompt: sentiment_probs tensor to list

def probs_to_list(tensor):
    return tensor.tolist()[0]

random_rows['sentiment_probs'] = random_rows['sentiment_probs'].apply(probs_to_list)


In [72]:
# prompt: argmax (neg	neu	pos)

import numpy as np

def argmax_sentiment(probs):
    return np.argmax(probs)

random_rows['argmax_sentiment'] = random_rows['sentiment_probs'].apply(argmax_sentiment)


In [74]:
random_rows['argmax_sentiment'] = random_rows['argmax_sentiment'].replace({0: 'positive', 1: 'negative', 2: 'neutral'})

In [75]:
# prompt: check acc label , argmax_sentiment

random_rows['check_acc_label'] = random_rows.apply(lambda row: 1 if row['Sentiment'] == row['argmax_sentiment'] else 0, axis=1)

accuracy = random_rows['check_acc_label'].sum() / len(random_rows)
print(f"Accuracy: {accuracy}")


Accuracy: 0.8666666666666667


In [76]:
random_rows

,Sentence,Sentiment,sentiment_probs,argmax_sentiment,check_acc_label
263,He joins Technopolis from KONE where he has he...,neutral,"[0.05938447639346123, 0.01415347121655941, 0.9...",neutral,1
1677,The Group 's revenue amounts to over EUR 80 mi...,neutral,"[0.061040882021188736, 0.012633203528821468, 0...",neutral,1
4157,The customer is local company Etera Mutual Pen...,neutral,"[0.03345048055052757, 0.01493827160447836, 0.9...",neutral,1
2277,`` Several growth initiatives in the chosen ge...,positive,"[0.7952315807342529, 0.006939348299056292, 0.1...",positive,1
1985,@Joerazorback This time I think $AAPL will mis...,negative,"[0.05498596653342247, 0.4583698809146881, 0.48...",neutral,0
4310,The new plant is planned to have an electricit...,neutral,"[0.17253173887729645, 0.007502424065023661, 0....",neutral,1
824,LONDON MarketWatch -- Nokia nok said it 's won...,positive,"[0.9459381699562073, 0.015258501283824444, 0.0...",positive,1
1109,"Progress Group , QPR 's representative in Saud...",positive,"[0.9384181499481201, 0.014653452672064304, 0.0...",positive,1
1221,They will be sunk to a depth of some 360-380 m...,neutral,"[0.029714807868003845, 0.02997729554772377, 0....",neutral,1
87,Metso Foundries Jyvaskyla Oy will discontinue ...,neutral,"[0.013046527281403542, 0.7316931486129761, 0.2...",negative,0


In [15]:
    def get_sentiment_label(compound_score):
        """
        แปลงค่า compound เป็นระดับอารมณ์ (Positive, Neutral, Negative)
        :param compound_score: ค่าคะแนน compound
        :return: ประเภทของอารมณ์ ('Positive', 'Neutral', 'Negative')
        """
        if compound_score > 0.05:
            return 0
        elif compound_score < -0.05:
            return 1
        else:
            return 2

In [18]:
random_rows["sentimentNLTK"] = random_rows["compound"].apply(get_sentiment_label)

KeyError: 'compound'

In [26]:
random_rows

,Label,Ticker,Headline,sentiment_probs,argmax_sentiment
2319,positive,WMT,India MPs in parliament uproar over Walmart l...,"[0.01895470917224884, 0.925135612487793, 0.055...",1
1437,positive,JNJ,Boris Johnson Secretly Asked For A Massive Amo...,"[0.03289394453167915, 0.03403576835989952, 0.9...",2
105,negative,AXP,Wikileaks lost more than $50 million of potent...,"[0.009829542599618435, 0.9507251381874084, 0.0...",1
2022,negative,TRV,Roughly 600000 travelers are stranded around t...,"[0.009013052098453045, 0.9405964612960815, 0.0...",1
850,negative,ADME,Dow plunges 531 points,"[0.02610127069056034, 0.7728283405303955, 0.20...",1
344,negative,BA,The angle-of-attack sensor that is believed to...,"[0.03980856016278267, 0.19033364951610565, 0.7...",2
1014,negative,AAAU,Leaked recording shows what Theresa May really...,"[0.02132164128124714, 0.715481162071228, 0.263...",1
1585,positive,MCD,"French McDonalds denies attack on ""cyber-glass...","[0.03993009403347969, 0.5522650480270386, 0.40...",1
536,positive,CVX,Chevron refuses to apologize in Ecuador pollut...,"[0.021847127005457878, 0.9292416572570801, 0.0...",1
1986,positive,CRM,Salesforce Named a Leader in CRM Suites by Ind...,"[0.8269126415252686, 0.009228994138538837, 0.1...",0


In [1]:
random_rows.loc[2861]['sentiment_probs']

NameError: name 'random_rows' is not defined

In [ ]:
# prompt: argmax_sentiment	sentimentNLTK check num of 2 col not equal

# Check if the number of rows where 'argmax_sentiment' and 'sentimentNLTK' are not equal
num_unequal = len(random_rows[random_rows['argmax_sentiment'] != random_rows['sentimentNLTK']])
print(f"Number of rows where 'argmax_sentiment' and 'sentimentNLTK' are not equal: {num_unequal}")


Number of rows where 'argmax_sentiment' and 'sentimentNLTK' are not equal: 3
